In [43]:
#Importing the libraries
import pandas as pd
import numpy as np
import requests
import json
import warnings

warnings.filterwarnings("ignore")

In [2]:
#SAving the headers
headers = {
        "X-RapidAPI-Key": "e6ccdd51b5msh9efa57294b05fc0p1230a0jsn7775531e3d9a",
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }

In [3]:
def get_airports():
    url = "https://aerodatabox.p.rapidapi.com/airports/search/term"
    word = input("Please enter the airport you want to search for: ")
    querystring = {"q": word, "limit": "10"}

    response = requests.get(url, headers=headers, params=querystring)
    airports = response.json()['items'] if 'items' in response.json() else 'No airport found'
    
    return airports


In [4]:
def get_codes():

    location = input('Insert location:')
    print()

    url = "https://aerodatabox.p.rapidapi.com/airports/search/term"
    querystring = {"q":location.lower(), "limit":"10"}

    response = requests.get(url, headers=headers, params=querystring)
    codes = response.json()['items'] if 'items' in response.json() else location

    return codes


def choose_code():

    codes = get_codes()

    while type(codes) is str or len(set([code['shortName'] for code in codes])) != 1:

        if not codes:
            print('Try another location. Unable to find code for:', codes, '\n')
        else:
            print('More than one location found. Insert one of the following airports:', [code['shortName'] for code in codes], '\n')

        codes = get_codes()

    return codes[0]['icao']


def flight_distance_time(from_, to_):
    url = f"https://aerodatabox.p.rapidapi.com/airports/icao/{from_}/distance-time/{to_}"
    response = requests.get(url, headers=headers)
    return pd.json_normalize(response.json()).T

In [50]:
#Creating a function to get the flight routes from the origin airport
def airport_routes(from_, date):

    #Gettig the basic information about the routes
    url = f"https://aerodatabox.p.rapidapi.com/airports/icao/{from_}/stats/routes/daily/{date}"
    response = requests.get(url, headers=headers)
    routes = response.json().get('routes', [])
    info = pd.json_normalize(routes).T

    #Adding the operator name to the dataframe extracting it from the operators JSON string
    if 'operators' in info.index:
        # Parsing the 'operators' JSON string into a Python dictionary
        operators = info.loc['operators'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
        # Checking if 'operators' is a list and if it has at least one item
        operator_name = operators.apply(lambda x: x[0]['name'] if isinstance(x, list) and len(x) > 0 else 'No operator found')
        operator_df = pd.DataFrame(operator_name).T
        operator_df.index = ['operator']
        #Adding the operator name to the dataframe
        info = pd.concat([info, operator_df])
        info.drop(index='operators', inplace=True)
        
    #If no operator is found, add a 'No operator found' string to the dataframe
    else:
        operator_df = pd.DataFrame(['No operator found']*len(info.columns), columns=info.columns, index=['operator'])
        info = pd.concat([info, operator_df])
        
    return info

In [47]:
#Testing the function
from_ = choose_code()
to_ = choose_code()
date = input("Please enter the date you want to search for using the format YYYY-MM-DD: ")
display(flight_distance_time(from_, to_))
display(airport_routes(from_, date))

,0
approxFlightTime,01:40:00
from.icao,LSZH
from.iata,ZRH
from.name,Zurich Kloten
from.shortName,Kloten
from.municipalityName,Zurich
from.location.lat,47.4647
from.location.lon,8.549169
from.countryCode,CH
to.icao,LECU


,0,1,2,3,4,5,6,7,8,9,...,218,219,220,221,222,223,224,225,226,227
averageDailyFlights,10.71,9.43,9.14,8.86,8.57,8.43,7.71,7.43,7.14,7.0,...,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14
destination.icao,EHAM,EDDB,EDDF,LOWW,LSGG,EDDH,LEMD,LFPG,LEBL,EDDL,...,GCLA,UTTT,LIMF,ENTO,LFTH,LICT,LIPX,EDLV,KPBI,NaN
destination.iata,AMS,BER,FRA,VIE,GVA,HAM,MAD,CDG,BCN,DUS,...,SPC,TAS,TRN,TRF,TLN,TPS,VRN,NRN,PBI,NaN
destination.name,Amsterdam Schiphol,Berlin Brandenburg,Frankfurt-am-Main,Vienna Schwechat,Geneva Cointrin,Hamburg,Madrid Adolfo Suárez –Barajas,Paris Charles de Gaulle,Barcelona,Duesseldorf Düsseldorf,...,"Sta Cruz de la Palma, La Palma Island La Palma",Tashkent,Torino Turin,Torp,Toulon/Hyères/Le Palyvestre Toulon-Hyères,Trapani Vincenzo Florio -Birgi,Verona Villafranca,Weeze Niederrhein,West Palm Beach Palm Beach,OPF
destination.shortName,Schiphol,Brandenburg,Frankfurt-am-Main,Schwechat,Cointrin,Hamburg,Adolfo Suárez –Barajas,Charles de Gaulle,Barcelona,Düsseldorf,...,La Palma,Tashkent,Turin,Torp,Toulon-Hyères,Vincenzo Florio -Birgi,Villafranca,Niederrhein,Palm Beach,NaN
destination.municipalityName,Amsterdam,Berlin,Frankfurt-am-Main,Vienna,Geneva,Hamburg,Madrid,Paris,Barcelona,Duesseldorf,...,"Sta Cruz de la Palma, La Palma Island",Tashkent,Torino,Torp,Toulon/Hyères/Le Palyvestre,Trapani,Verona,Weeze,West Palm Beach,NaN
destination.location.lat,52.3086,52.35139,50.0264,48.1103,46.2381,53.6304,40.4936,49.0128,41.2971,51.2895,...,28.6265,41.2579,45.2008,59.1867,43.0973,37.9114,45.3957,51.6024,26.6832,NaN
destination.location.lon,4.763889,13.493889,8.543129,16.5697,6.10895,9.988229,-3.56676,2.549999,2.078459,6.766779,...,-17.755598,69.2812,7.64963,10.2586,6.146029,12.487999,10.8885,6.142169,-80.0956,NaN
destination.countryCode,NL,DE,DE,AT,CH,DE,ES,FR,ES,DE,...,ES,UZ,IT,NO,FR,IT,IT,DE,US,NaN
destination.localCode,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,ТАС,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
